# Function as the first-class object
-----------------

Luciano Ramalho. **Fluent Python**

**“first-class object”** is a program entity that can be:
- created at runtime
- assigned to a variable or element in a data structure
- passed as an argument to a function
- returned as the result of a function

### 1. Callable Objects
-------------
* Object is callable if the **call operator** ``()`` may be applied to it
* Built-in ```callable() ``` returns ``` True ``` if an object is callable
* Not only are Python functions real objects, but arbitrary Python objects may also be made to behave like functions

In [ ]:
[callable(obj) for obj in (abs, str, 13)]

#### 1.1. Callable types in Python Data Model
----------------
1. User-defined functions -- created with ``def`` statements or ``lambda`` expressions
2. Built-in functions  -- implemented in C (for CPython)
3. Built-in methods -- functions, implemented in C (like ``dict.get``)
4. Methods -- functions defined in the body of a class
5. Classes -- when invoked
   *  a class runs its
   
      1)  ``__new__ `` method to create an instance
      
      2)  ``__init__`` to initialize it 
   * finally the instance is returned to the caller
   * there isn't ``new`` operator, so calling a class is like calling a function
6. Class instances -- if a class defines a **``__call__``** method, then its instances may be invoked as functions
7. Generator functions -- functions or methods that use ``yield`` statement; when called, generator functions return a generator object.

#### 1.2. Function-like objects
-----------------

* A class implementing ``__call__`` is an easy way to create function-like objects that have some **internal state** that must be kept across invocations

* A closure --  function with internal state defined by **attributes**  

In [ ]:
import random

class BingoCage:

    def __init__(self, items):
        self.items = items  
    
    @property
    def items(self):
        return self._items

    @items.setter
    def items(self, value): 
        self._items = list(value) #internal state defined by attribute
        random.shuffle(self.items)
        
    def pick(self):  
        try:
            return self.items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')  

    def __call__(self, a):
        print(f'attempt={a}')
        return self.pick()
    

In [ ]:
bingo = BingoCage(range(5))

In [ ]:
print(bingo.items)

In [ ]:
type(bingo.items)

In [ ]:
bingo.pick()

In [ ]:
print(bingo.items)

In [ ]:
bingo(1)

In [ ]:
print(bingo.items)

In [ ]:
bingo(2)

In [ ]:
print(bingo.items)

In [ ]:
bingo(4)

#### 1.3. Function as callable: Introspection
-----------------
Additional attributes of function-like objects


In [ ]:
def f(): pass

In [ ]:
dir(f)

In [ ]:
class C: pass

In [ ]:
obj=C()
dir(obj)

In [ ]:
set(dir(f)) - set(dir(obj))

In [ ]:
print(set(dir(f)) - set(dir(obj)))

| name | type | description |
|--------------|-----------|-----------|
|``__annotations__ ``|``dict`` | parameter and return annotations|
|**``__call__``**| method-wrapper | implementation of the () operator|
|**``__closure__``**|``tuple``| the function closure, i.e. bindings for **free variables** (often is None)|
|``__code__`` | code | function metadata and function body compiled into bytecode|
|``__defaults__``| ``tuple``| default values for the formal parameters|
|``__get__ ``|method-wrapper| implementation of the read-only descriptor protocol|
|``__globals__``|`` dict ``| global variables of the module where the function is defined|
|``__kwdefaults__``|`` dict``| default values for the keyword-only formal parameters|
|``__name__``|`` str ``|the function name|
|``__qualname__``|`` str``| the qualified function name, ex.: Random.choice|

#### 1.4. Retrieving information about parameters
-----------------

In [ ]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [ ]:
t='123 456 789'
clip(t,5)

* Default values for the formal parameters

In [ ]:
clip.__defaults__

In [ ]:
clip.__kwdefaults__

* Parameters and local variables, does not include any variable arguments prefixed with ``*`` or ``**``

In [ ]:
clip.__code__.co_varnames 

* Using ``inspect.signature()``

In [ ]:
from inspect import signature
sig = signature(clip)
sig

In [ ]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

Besides ``name``, ``default`` and ``kind``, ``inspect.parameter`` objects have an annotation attribute
which is usually ``inspect._empty`` but may contain function signature metadata
provided via the annotations syntax

* Mapping the passed `args`and `kwargs` to the function's signature

``sig.bind(*args, **kwargs)`` -- get a BoundArguments object for mapping; the same machinery the interpreter uses to bind arguments to formal parameters in function calls 

* Frameworks and tools like IDEs can use this information to validate code

#### 1.5. Function annotations
-------------------

* Each argument in the function declaration may have an **annotation expression**
    * preceded by `:` 
    * if there is a default value, the annotation goes between the argument name and the `=` sign
    * to annotate the return value, add ``->`` and expression (may be of any type) between the `)` and the `:` at the tail of the function declaration 
* the most common types used in annotations are classes, like ``str`` or ``int``, or strings, like ``'int> 0'``
* annotations **have no meaning** to the interpreter; they are just metadata that may be used by tools, such as IDEs, frameworks and decorators
* ``inspect.signature()`` knows how to extract the annotations

In [ ]:
def clip(text:str, max_len:'int > 0'=80) -> str:  
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [ ]:
clip.__annotations__

## 2. Decorators
-----------------
[Corey Schafer:Decorators - Dynamically Alter The Functionality Of Your Functions](https://www.youtube.com/watch?v=FsAPt_9Bf3U&ab_channel=CoreySchafer)


A function decorator 
* is a **callable** that takes another function in the source code as argument ( **decorated function**) to enhance it behavior is some way:
    * to perform a particular action before and\or after executing each of the functions
    * to pass in an extra parameter
    * to convert the output to another format with another function or callable object
* is based on the **closure** data

#### 2.1. Closure concept
___________________

* first-class functions allow treating functions like any other object
* inner function accesses args of outer function and **free** variables (which are not actually defined in inner function)
* after execution of ``outer_func`` the inner function still has access to the free variables

A closure is an inner function that remembers and has access to variables in the local scope in which it was created even after the outer function has finished executing

In [ ]:
def outer_func(msg):
    message = msg
    
    def inner_func():
        print(message)
        
    return inner_func

In [ ]:
hi_func = outer_func('Hi')
hello_func = outer_func('Hello')

In [ ]:
print(hi_func)

In [ ]:
print(hi_func.__name__)

In [ ]:
hi_func()
hello_func()

In [ ]:
# equivalent definition
def outer_func1(msg):   
    def inner_func():
        print(msg)
        
    return inner_func

In [ ]:
hi_func = outer_func1('Hi')
hello_func = outer_func1('Hello')

In [ ]:
hi_func()
hello_func()

In [ ]:
hi_func.__code__.co_varnames

In [ ]:
hi_func.__code__.co_freevars

In [ ]:
hi_func.__closure__

In [ ]:
hi_func.__closure__[0].cell_contents

In [ ]:
hello_func.__closure__[0].cell_contents

In [ ]:
outer_func.__closure__

In [ ]:
dir(outer_func)

In [ ]:
type(hi_func.__closure__)

#### 2.2. Function decorator concept
___________________

In [ ]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f'Executed before {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'Executed after {original_function.__name__} \n')
        return result
    return wrapper_function

In [ ]:
@decorator_function
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

In [ ]:
display_info('John', 25)

In [ ]:
display_info('Travis', 30)

#### 2.3. Class decorator concept
___________________
* ``__init__(self, original_function)`` -- takes an original_function
* ``__call__(self, *args, **kwargs)`` -- as a wrapper function

In [ ]:
class decorator_class:
    def __init__(self, original_function):
        self.original_function = original_function
    def __call__(self, *args, **kwargs):
        print(f'Executed before {self.original_function.__name__}')
        return self.original_function(*args, **kwargs)

In [ ]:
@decorator_class
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

In [ ]:
display_info('John', 25)

In [ ]:
display_info('Travis', 30)

#### 2.4. Decoration process
___________________

* Decorators run **right after** the decorated function is defined (a **key feature** of decorators)

In [ ]:
registry = []  

def register(func):  
    print(f'running register({ func})')
    registry.append(func) 
    return func  

In [ ]:
@register  
def f1():
    print('running f1()')

In [ ]:
@register
def f2():
    print('running f2()')

In [ ]:
def f3():  
    print('running f3()')

In [ ]:
def main(): 
    print('running main()')
    print('registry:', registry)
    f1()
    f2()
    f3()

In [ ]:
main() 

In [ ]:
for f in registry:
    f()